In [1]:
from tqdm import tqdm
from time import sleep
import requests
import re
import json
import pandas as pd
import csv
from selenium.webdriver import ActionChains
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait as wait

from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
import time

In [2]:
def google_reviews(stroe_list,gu,count=10):
    
    #크롬 드라이버 실행
    
    chrome_options = Options()
    chrome_options.add_experimental_option("detach",True)
    chrome_options.add_experimental_option("excludeSwitches",["enable-logging"])

    service = Service(executable_path=ChromeDriverManager().install())
    driver=webdriver.Chrome(service=service, options=chrome_options)
    driver.get('https://www.google.com/maps/?hl=en-US')
    driver.implicitly_wait(10)
    
    #리스트 형식으로 저장된 가게 하나씩 검색
    for store in store_list:
        count=count
        driver.get('https://www.google.com/maps/?hl=en-US') #구글맵 영어 창 열기
        #result_list=[]
        time.sleep(3)
        
        #검색창에 검색
        search=driver.find_element(By.CSS_SELECTOR,'#searchboxinput') #검색창 열기
        search.send_keys(store+' '+gu+'\n') #주소 + 장소이름 검색
        
        #가게 확인
        #time.sleep(5)
        #check=input('일치하는 가게를 클릭 후 y를 눌러주세요(이외 멈춤)')
        #if check.upper()=='Y':
            #now_url=driver.current_url
        #else:
         #   break
            
        #리뷰더보기로 이동
   

        review_btn=wait(driver, 10).until(EC.element_to_be_clickable((By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/div/div[1]')))
        #review_btn=driver.find_element(By.CSS_SELECTOR,'.mgr77e')
        review_btn.click()
        views=int(''.join(review_btn.text[4:-7].split(',')))-1
        print(views)
        
       #div태그 스크롤
        for i in range(2000):
            try:
                scroll = driver.find_element(By.CSS_SELECTOR,'div.m6QErb.DxyBCb.kA9KIf.dS8AEf')
                scroll.send_keys(Keys.PAGE_DOWN)
                if i/10==0:
                    time.sleep(1)
                #location1=scroll.location_once_scrolled_into_view
                #print(i, location1)
            
            except Exception as e:
                print(e)
                break
        
        
        
        reviews = driver.find_elements(By.CSS_SELECTOR,'.jftiEf')
        name=[]
        star=[]
        content=[]
        date=[]
        for review in reviews:
            try:
                more_btn = review.find_element(By.CSS_SELECTOR,'.w8nwRe.kyuRq').click()
                print("y",end="")
                WebDriverWait(review,60).until(EC.invisibility_of_element_located((By.CSS_SELECTOR,".w8nwRe.kyuRq")))
                name.append(review.find_element(By.CSS_SELECTOR,'.d4r55').text)
                star.append(int(float(review.find_element(By.CSS_SELECTOR,'.kvMYJc').get_attribute("aria-label")[1])))
                content.append(review.find_element(By.CSS_SELECTOR,'.wiI7pd').text.replace("\n"," "))
                date.append(review.find_element(By.CSS_SELECTOR,'.rsqaWe').text)
            except:
                print("n",end="")
                name.append(review.find_element(By.CSS_SELECTOR,'.d4r55').text)
                star.append(int(float(review.find_element(By.CSS_SELECTOR,'.kvMYJc').get_attribute("aria-label")[1])))
                content.append(review.find_element(By.CSS_SELECTOR,'.wiI7pd').text.replace("\n"," "))
                date.append(review.find_element(By.CSS_SELECTOR,'.rsqaWe').text)
        
        emoji = re.compile("["
                    u"\U0001F600-\U0001F64F"
                    u"\U0001F300-\U0001F5FF"
                    u"\U0001F680-\U0001F6FF"
                    u"\U0001F1E0-\U0001F1FF"
                       "]+", flags=re.UNICODE)
        
        content_noemj=[]
        name_noko=[]
        
        for i in name :
            i = re.sub('([ㄱ-ㅎㅏ-ㅣ]+)', 'KOREAN', i)
            i = i.encode('utf-8').decode('ascii','ignore')
            name_noko.append(i)
            
        for i in content :
            i = emoji.sub(r'',i)
            i = re.sub('[-=+,#/\?:^$.@*\"※~&%ㆍ!』\\‘|\(\)\₩[\]\<\>`\'…《\》’]', '', i)
            i = re.sub('([ㄱ-ㅎㅏ-ㅣ]+)', '', i) 
            i = i.encode('utf-8').decode('ascii','ignore')
            content_noemj.append(i)
            
        print(len(content),len(name))
        all=pd.DataFrame([name_noko,star,content_noemj,date],index=['NAME','STAR','CONTENT','DATE']).transpose()
        print(content_noemj)
        file_name = '{}review.csv'.format(store+' ')
        all.to_csv(file_name)
    
                


In [3]:
store_list=['Deoksugung','War Memorial Korea','yeouido hangang park']
google_reviews(store_list,'Seoul',5)

16120
nnnnynnnnynnnnnnnnnnnnynnnnnnynnnnynynnnnnnynynnnnynynynnnnnnnnnnynynnnnnnnynnnnnnnnynnnnnnnnnnnnnynnnnynynynnnnnnnnnnnnnnnynnynnnnnnnnnnnnynnnnnnnnnnnnnnnnnnnynnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnynnynnnnnnnnnnnynnnnynnnnnnnnnnynnnnnnnnnnnynnnnnnnnnnnnnnnnnnnnnynnynnnnnnynynnnnnnnnnnnnnynynnnnnnnnnnynnnynnnnynynnnynnnnnnnnnnnnnnynnnynynnnnnynnnnnnynnnnnnnnnynynnnnnnnnnnnnnnnnnnnnnnnnynnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnnn930 930
['One of